In [33]:
from pyspark.sql import SparkSession

In [34]:
spark = SparkSession.builder.appName('Week4Practice').getOrCreate()

In [35]:
df = spark.read.csv('../../Week4/datasets/movie_reviews.csv', header=True, inferSchema=True)

In [36]:
df.show(5)

+------+-------+------+------------------+
|userId|movieId|rating|            review|
+------+-------+------+------------------+
|    92|    176|   4.6|  Highly recommend|
|    30|     57|   2.5|           Not bad|
|    62|     64|   2.2|Average experience|
|    64|    173|   2.1|Average experience|
|    49|     44|   3.5|  Highly recommend|
+------+-------+------+------------------+
only showing top 5 rows


In [37]:
df.printSchema()

root
 |-- userId: integer (nullable = true)
 |-- movieId: integer (nullable = true)
 |-- rating: double (nullable = true)
 |-- review: string (nullable = true)



In [38]:
train, test = df.randomSplit([0.75, 0.25])

In [39]:
from pyspark.ml.recommendation import ALS

In [51]:
als = ALS(userCol="userId", itemCol="movieId", ratingCol="rating",rank=10, maxIter=10, regParam=0.1, coldStartStrategy="drop", nonnegative=True)
# passing coldSatrtStrategy is imp deafault value is 'nan' whole Rmse will be fucked
model = als.fit(train)

In [60]:
predictions = model.transform(test)
predictions.show()

+------+-------+------+------------------+----------+
|userId|movieId|rating|            review|prediction|
+------+-------+------+------------------+----------+
|    31|     51|   1.6|   Could be better| 2.9769402|
|    31|    139|   1.0|          Terrible| 2.5144107|
|    31|    189|   4.8|  Highly recommend|  1.269039|
|    85|     29|   4.5|  Highly recommend| 2.9175968|
|    53|     94|   2.4|Average experience| 1.9511662|
|    53|     99|   2.0|          Loved it| 3.0233846|
|    34|     11|   2.2|     Waste of time|0.89292973|
|    34|     88|   2.6|           Not bad|  2.092395|
|    34|    102|   3.1| Amazing storyline|  2.903771|
|    34|    178|   2.6|Average experience|  4.347064|
|    34|    190|   3.1|       Good acting| 2.6881213|
|    81|    153|   2.8|           Not bad|   2.20012|
|    81|    174|   4.9|          Loved it| 2.6346138|
|    28|     80|   2.6| Amazing storyline| 3.0419924|
|    28|    116|   4.9|   Could be better| 1.0960867|
|    76|    177|   3.4|     

In [61]:
from pyspark.ml.evaluation import RegressionEvaluator

In [62]:
evaluator = RegressionEvaluator(metricName="rmse", labelCol="rating", predictionCol="prediction")

In [63]:
evaluator.evaluate(predictions)

1.5025764380819233